In [63]:
import pandas as pd
import mplfinance as mpf
import requests
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as patches
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go 

#url = "https://api.twelvedata.com/time_series?symbol=SOL&interval=5min&outputsize=90&apikey=b1f81ac90e5d4297bc5a4e3704a79c31&format=JSON"
url = "https://api.twelvedata.com/time_series?symbol=AAPL&interval=5min&outputsize=90&apikey=b1f81ac90e5d4297bc5a4e3704a79c31&format=JSON"
#url = "https://api.twelvedata.com/time_series?symbol=BTC&interval=5min&apikey=b1f81ac90e5d4297bc5a4e3704a79c31&start_date=2025-03-24"
response = requests.get(url)
data = response.json()
copy_data = response.json()
candles = data["values"][:90][::-1] 

df = pd.DataFrame(candles)
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime' , inplace=True)
df = df.sort_index()
for col in ['open','high','low','close','volume']:
     df[col] = df[col].astype(float)


In [ ]:
import backtrader as bt

class Strategy(bt.Strategy):
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def __init__(self):
        pass

    def next(self):
        if len(self)<60:
            return 





        trend = self.isBearishorBullish()
        fvg_zones, fvg_places, fvg_datetime, testedFvgs = self.FVG()

        print("Is Bullish:", trend)
        print("FVG Zones:", fvg_zones)
        print("FVG Datetime:", fvg_datetime)
        print("Tested FVGs:", testedFvgs)

    def FVG(self):
        is_bullish = self.isBearishorBullish()
        fvg_zones = []
        fvg_places = []
        fvg_datetime = []

        
        lookback = 60
        if len(self)<lookback:
            return
        

        for t in range(-lookback, -3):  
            try:
                if is_bullish ==True:
                    high1 = self.datas[0].high[t]
                    low3 = self.datas[0].low[t + 2]
                    if float(high1) < float(low3):  # Bullish FVG
                        fvg_zones.append((float(high1), float(low3)))
                        fvg_places.append(t)
                        fvg_datetime.append(self.datas[0].datetime.date(t))
                elif is_bullish == False:
                    low1 = self.datas[0].low[t]
                    high3 = self.datas[0].high[t + 2]
                    if float(low1) > float(high3):  # Bearish FVG
                        fvg_zones.append((float(high3), float(low1)))
                        fvg_places.append(t)
                        fvg_datetime.append(self.datas[0].datetime.date(t))
            except IndexError:
                continue

        testedFvgs = self.IsFVGTested(fvg_places, is_bullish)
        return fvg_zones, fvg_places, fvg_datetime, testedFvgs

    def IsFVGTested(self, fvg_places, is_bullish):
        tested_fvg = []
        data_len = len(self.datas[0])
        lookback=60
        

        for t in fvg_places:
            if t + 3 >= lookback:
                continue

            try:
                if is_bullish:
                    fvg_low = float(self.datas[0].low[t + 2])
                    fvg_high = float(self.datas[0].high[t])
                    for i in range(t + 3, -lookback,-1):
                        candle_low = float(self.datas[0].low[i])
                        candle_high = float(self.datas[0].high[i])
                        if candle_low <= fvg_low <= candle_high:
                            tested_fvg.append((fvg_low, fvg_high))
                            break
                else:
                    fvg_low = float(self.datas[0].high[t + 2])
                    fvg_high = float(self.datas[0].low[t])
                    for i in range(t + 3, -lookback,-1):
                        candle_low = float(self.datas[0].low[i])
                        candle_high = float(self.datas[0].high[i])
                        if candle_low <= fvg_high and candle_high >= fvg_low:
                            tested_fvg.append((fvg_low, fvg_high))
                            break
            except IndexError:
                continue

        return tested_fvg

    def isBearishorBullish(self):
        up_count = 0
        down_count = 0
        lookback=60
        if len(self.datas[0]) < lookback + 1:
            return
         
        for i in range(-lookback, -1):
            prev_close = self.datas[0].close[i]
            
            curr_close = self.datas[0].close[i + 1]
            
            if float(curr_close) > float(prev_close):
                
                up_count += 1
            else:
                down_count += 1

        if up_count > down_count:
            
            return True
        elif down_count > up_count:
            
            return False
        else:
            
            return None


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90 entries, 2025-07-10 15:00:00 to 2025-07-11 15:55:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    90 non-null     float64
 1   high    90 non-null     float64
 2   low     90 non-null     float64
 3   close   90 non-null     float64
 4   volume  90 non-null     float64
dtypes: float64(5)
memory usage: 4.2 KB


In [70]:
data = bt.feeds.PandasData(dataname=df,
                           high=1,
                           low=2,
                           open=0,
                           close=3,
                           volume=4,
                           openinterest=-1
                           )
 
# -------------- BACKTEST ----------------
cerebro = bt.Cerebro()
cerebro.addstrategy(Strategy)
cerebro.adddata(data)
cerebro.broker.set_cash(10000)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
print(f"Starting Portfolio Value: {cerebro.broker.getvalue()}")
cerebro.run()
print(f"Final Portfolio Value: {cerebro.broker.getvalue()}")

Starting Portfolio Value: 10000
Close[-5]: 211.105
Close[-4]: 211.17
Close[-3]: 211.0401
Close[-2]: 211.12
Close[-1]: 211.020004
the data length is 60
Is Bullish: None
FVG Zones: []
FVG Places: []
FVG Datetime: []
Tested FVGs: []
Close[-5]: 211.17
Close[-4]: 211.0401
Close[-3]: 211.12
Close[-2]: 211.020004
Close[-1]: 211.16499
FALSE down 31 up 28
FALSE down 31 up 28
the data length is 61
Is Bullish: False
FVG Zones: [(212.62, 212.82001), (211.03, 212.089996), (210.87, 212.17), (211.67, 211.73), (211.050003, 211.094696), (210.87, 210.89999), (210.63, 210.8488), (210.67, 210.75999)]
FVG Places: [-52, -50, -49, -37, -33, -30, -22, -14]
FVG Datetime: [datetime.date(2025, 7, 10), datetime.date(2025, 7, 10), datetime.date(2025, 7, 10), datetime.date(2025, 7, 11), datetime.date(2025, 7, 11), datetime.date(2025, 7, 11), datetime.date(2025, 7, 11), datetime.date(2025, 7, 11)]
Tested FVGs: [(212.62, 212.82001), (211.03, 212.089996), (210.87, 212.17), (211.67, 211.73), (211.050003, 211.094696), (